In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm
import scipy.linalg
import math

import torch

def power_of_matrix(A, n):
    """
    Calculate A^n with A matrix
    """
    if n == 0:
        res = np.eye(A.shape[0])
        return res
    res = A
    while n > 1:
        res = res @ A
        n -= 1
    return res

def solution(U):
    # find the eigenvalues and eigenvector of U(transpose).U
    e_vals, e_vecs = np.linalg.eig(np.dot(U.T, U))  
    # extract the eigenvector (column) associated with the minimum eigenvalue
    return e_vecs[:, np.argmin(e_vals)] 

In [ ]:
# Value function for classical market making problem, calculate v(., T) / T
Time = np.arange(50, 950, 10)

dt = 1
# Posted LO is filled with probability of e^(-kappa*depth)
kappa_sell , kappa_buy = 10**1 , 10**1
assert kappa_buy == kappa_sell
kappa = kappa_sell
# Arrival rate of selling MOs from other participants in market
lambda_sell = 1
# Arrival rate of buying MOs from other participants in market
lambda_buy = 1
# Inventory boundary
q_upper = 30
q_lower = -30
# Risk aversion parameters
alpha = 0
phi = 1*10**-5

sigma = 0.05


X0 = 0
S0 = 10

# Let A denote a square matrix
A = np.zeros([q_upper-q_lower+1, q_upper-q_lower+1])
z = np.zeros(q_upper-q_lower+1)
# w = np.zeros([length, q_upper - q_lower + 1])
# optimal_depth_bid = np.zeros([length, q_upper - q_lower])
# optimal_depth_ask = np.zeros([length, q_upper - q_lower])

# Notice we assume kappa_buy = kappa_sell in the model
# i denotes the column number
for i in range(q_upper-q_lower+1):
    # j denotes the row number
    for j in range(q_upper-q_lower+1):
        if j == i:
            # q = q_upper - j
            A[j,i] = - phi*kappa*(q_upper-j)**2
        elif j == i-1:
            A[j,i] = lambda_buy*np.e**(-1)
        elif j == i+1:
            A[j,i] = lambda_sell*np.e**(-1)

for i in range(q_upper-q_lower+1):
    # q = q_upper - i
    z[i] = np.exp(-alpha*kappa*(q_upper-i)**2)

In [ ]:
Q_initial = [-15, -10, -5, -1, 3, 5]

value_over_t = []

for Q0 in Q_initial:
    value_over_t_q0 = []
    for T in Time: 
        t =np.arange(0, T+dt, dt)
        length = len(t)

        h = np.zeros([length , q_upper-q_lower+1])
        delta_buy = np.zeros([length , q_upper-q_lower])
        delta_sell = np.zeros([length , q_upper-q_lower])
            
        for i in range(length):
            h[i] = np.log(np.dot(expm(A*(T-i*dt)),z))/kappa

        for j in range(q_upper-q_lower):
            # q = q_upper - j
            delta_sell[:,j] = 1/kappa + h[:,j] - h[:,j+1]

        for j in range(q_upper-q_lower):
            # q = q_upper -1 - j
            delta_buy[:,j] = 1/kappa + h[:,j+1] - h[:,j]

        theo_value_function = h[0,int(q_upper - Q0)]
        value_over_t_q0.append(theo_value_function / T)
    value_over_t.append(value_over_t_q0)

In [ ]:
# Let see whether A is diagonalizable
import scipy.linalg

evalues, evectors = scipy.linalg.eig(A)
Tau = np.zeros((len(evalues), len(evalues)))

for i in range(len(evalues)):
    Tau[i, i] = np.real(evalues[i])

gamma = np.real(max(evalues)) / kappa_sell

In [ ]:
for i in range(len(Q_initial)):
    plt.plot(Time, value_over_t[i], label=f"Q0={Q_initial[i]}")
plt.plot(Time, [gamma for i in Time], '--', label='$\lambda_{max} / \kappa$')
plt.xlabel("Terminal Time $T$")
plt.ylabel("$v(t=0,q; T) / T$")
plt.title("Value of $v(t=0, q; T) / T$ as T increases")
plt.legend()
plt.show()

## Numerically solve the ergodic control problem

In [ ]:
# Let C denote a square matrix
C = np.zeros([q_upper-q_lower+1, q_upper-q_lower+1])

# Notice we assume kappa_buy = kappa_sell in the model
# i denotes the column number
for i in range(q_upper-q_lower+1):
    # j denotes the row number
    for j in range(q_upper-q_lower+1):
        if j == i:
            # q = q_upper - j
            C[j,i] = - kappa*(phi*(q_upper-j)**2 + gamma)
        elif j == i-1:
            C[j,i] = lambda_buy*np.e**(-1)
        elif j == i+1:
            C[j,i] = lambda_sell*np.e**(-1)

In [ ]:
print("The shape of coefficient matrix is:", C.shape)
print("The rank of coefficient matrix is:", np.linalg.matrix_rank(C))
print("Therefore, there will be non-trivial solution of v(q)")
print("Since the rank of coefficient matrix is n-1, so the solution must be unique up to a constant")

In [ ]:
Q = torch.arange(q_lower,q_upper+1,1).type(torch.float)
Q = Q.reshape(-1,1)
Q = Q.detach().cpu().numpy()

solu1 = -1 * solution(C)
solu2 = -5 * solution(C)
solu3 = -10 * solution(C)

plt.plot(Q.flatten(), np.log(solu1)/kappa, label='solution 1')
plt.plot(Q.flatten(), np.log(solu2)/kappa, label='solution 2')
plt.plot(Q.flatten(), np.log(solu3)/kappa, label='solution 3')

plt.legend()
plt.xlabel("q")
plt.ylabel("value")
plt.title("Solutions $\hat{v}(q)$ to Ergodic HJB Equation")
plt.show()

In [ ]:
solu2 = np.log(solu2)/kappa

delta_buy = np.zeros(q_upper - q_lower)
delta_sell = np.zeros(q_upper - q_lower)

for i in range(1, q_upper-q_lower+1):
    delta_sell[i-1] = 1/kappa_buy - solu2[i-1] + solu2[i]
for i in range(q_upper-q_lower):
    delta_buy[i] = 1/kappa_buy + solu2[i] - solu2[i+1]

In [ ]:
plt.plot(Q.flatten()[1:], delta_sell, 'o',label='sell depth $\delta^{+}$')
plt.plot(Q.flatten()[:-1], delta_buy, 'o',label='buy depth $\delta^{-}$')
plt.xlabel("q")
plt.ylabel('$\delta^{\pm}(\$)$')
plt.title("Optimal Strategy $\delta^{\pm, *}(q)$ for Ergodic Control Problem")
plt.legend()
plt.show()

## Ergodicity

In [ ]:
T = 2000
ts_size = 0.1
ts = np.arange(0, T+ts_size, ts_size)

Cash = []
Inventory = []
Midprice = []

length = len(ts)

for _ in range(5000):
    X = np.zeros(length)
    Q = np.zeros(length)
    S = np.zeros(length)
    dW = np.sqrt(dt)*np.random.randn(length-1)
    X[0] = 0
    Q[0] = 0
    S[0] = S0

    for i in range(length-1):
        S[i+1] = S[i] + sigma * dW[i]
        # current position
        q = int(Q[i])

        delta_sell_current = 1e20 if q <= q_lower else delta_sell[q_upper+q-1]
        delta_buy_current = 1e20 if q >= q_upper else delta_buy[q_upper+q]

        dN_sell = np.random.poisson(lambda_buy*np.exp(-kappa*delta_sell_current)*ts_size)
        dN_buy = np.random.poisson(lambda_sell*np.exp(-kappa*delta_buy_current)*ts_size)
        # Cash process
        X[i+1] = X[i] + (S[i]+delta_sell_current)*dN_sell - (S[i]-delta_buy_current)*dN_buy
        # Inventory process
        Q[i+1] = Q[i] + dN_buy - dN_sell

    # calculate value function
    Cash.append(X)
    Inventory.append(Q)
    Midprice.append(S)

Cash = np.array(Cash)
inv = np.array(Inventory)
Midprice = np.array(Midprice)

mtm = Cash + inv * Midprice

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(17,8))

ax[0,0].plot(ts, np.mean(inv, axis=0))
ax[0,0].set_xlabel("t(s)")
ax[0,0].set_ylabel("$E[Q_t]$")

ax[1,0].plot(ts, np.mean(inv**2, axis=0)**0.5)
ax[1,0].set_xlabel("t(s)")
ax[1,0].set_ylabel("$E[Q_t^2]^{1/2}$")

a = np.mean(inv**3, axis=0)
ax[0,1].plot(ts, np.sign(a) * (np.abs(a)) ** (1 / 3))
ax[0,1].set_xlabel("t(s)")
ax[0,1].set_ylabel("$E[Q_t^3]^{1/3}$")

ax[1,1].plot(ts, np.mean(inv**4, axis=0)**(1/4))
ax[1,1].set_xlabel("t(s)")
ax[1,1].set_ylabel("$E[Q_t^4]^{1/4}$")

a =  np.mean(inv**5, axis=0)
ax[0,2].plot(ts, np.sign(a) * (np.abs(a)) ** (1 / 5))
ax[0,2].set_xlabel("t(s)")
ax[0,2].set_ylabel("$E[Q_t^5]^{1/5}$")

ax[1,2].plot(ts, np.mean(inv**6, axis=0)**(1/6))
ax[1,2].set_xlabel("t(s)")
ax[1,2].set_ylabel("$E[Q_t^6]^{1/6}$")

fig.suptitle('$\mathbb{E}[Q_t^p]^{1/p}$ over time starting with $Q_0 = 0$', fontsize=15)

plt.show()

In [ ]:
# Let A denote a square matrix
P = np.zeros([q_upper-q_lower+1, q_upper-q_lower+1])

# Notice we assume kappa_buy = kappa_sell in the model
# i denotes the row number
for i in range(1, q_upper-q_lower):
    # j denotes the column number
    for j in range(q_upper-q_lower+1):
        if j == i:
            # q = q_upper - j
            P[i,j] = 1 - lambda_buy*np.e**(-kappa*delta_sell[q_upper - q_lower - 1 - i]) - lambda_sell*np.e**(-kappa*delta_buy[q_upper - q_lower - i])
        elif j == i+1:
            P[i,j] = lambda_buy*np.e**(-kappa*delta_sell[q_upper - q_lower - 1 - i])
        elif j == i-1:
            P[i,j] = lambda_sell*np.e**(-kappa*delta_buy[q_upper - q_lower - i])


In [ ]:
P[0,0] = 1 - lambda_buy*np.e**(-kappa*delta_sell[q_upper - q_lower - 1])
P[0,1] = lambda_buy*np.e**(-kappa*delta_sell[q_upper - q_lower - 1])
P[q_upper - q_lower,q_upper - q_lower] = 1 - lambda_sell*np.e**(-kappa*delta_buy[0])
P[q_upper - q_lower,q_upper - q_lower-1] = lambda_sell*np.e**(-kappa*delta_buy[0])

In [ ]:
# Let see whether A is diagonalizable
evalues, evectors = scipy.linalg.eig(P)
Tau = np.zeros((len(evalues), len(evalues)))

for i in range(len(evalues)):
    Tau[i, i] = np.real(evalues[i])

In [ ]:
z = np.zeros(q_upper-q_lower+1)
z[q_upper] = 1
K = np.zeros([q_upper-q_lower+1, q_upper-q_lower+1])
K[0,0] = 1

pi = z @ evectors @ K @ scipy.linalg.inv(evectors)

In [ ]:
fig, ax = plt.subplots()

ax.hist([inv[:, 1050], inv[:, 1200], inv[:, 1500],inv[:, 1750], inv[:, 2000]],bins=21,
          alpha=0.5,label=['t=1000s','t=1250s','t=1500s','t=1750s','t=2000s'])
ax.legend()
ax.set_xlabel("q")
ax.set_ylabel("Frequency")
ax.set_title("Distribution of $Q_t$ over time t")

ax2=ax.twinx()
ax2.plot(np.arange(q_lower,q_upper+1,1), pi, '--', label='theoretical distribution')
ax2.legend(loc=2)
ax2.set_ylim(0, 0.08)
ax2.set_ylabel('Density')
plt.show()